# Filtering 1

In this lecture, we first cover lambda functions, which are a more concise way to define functions.  Lambda functions come in handy when you want to use apply or map to create a custom column as a function of other columns.  Then we will see how to select only rows of a given pandas data frame that satisfy a certain condition(s).  This lecture concludes with a series of helpful miscellaneous pandas tools.

## Lambda Functions

Below I give examples of functions defined with a def statement and then the equivalent function defined with a lambda statement.  All lambda function begin with the keyword lambda followed by the various inputs and then a colon.  Following the colon is what you want to return:

    lambda inputs: return 

In [1]:
def add_five(x):
    """Adds 5 to the input x"""
    
    return x+5

#We call the function through the function name
add_five(10)

15

In [2]:
#Equivalent lambda function
f = lambda x: x+5

#We call the function through the variable that we store the lambda function in
f(10)

15

Next let's see an example where we have multiple inputs in the function

In [3]:
def Make_Name(first,last):
    """Takes as input a first and last name
    and combines them into a single name with a space
    in between"""
    
    
    return first + " " + last

#Call the function
Make_Name("Jon", "Smith")

'Jon Smith'

In [4]:
#Equivalent function with lambda
f = lambda first,last: first + " " + last

#Call the function
f("Jon", "Smith")

'Jon Smith'

Finally, let's see how we can use an if else statement with a lambda

In [6]:
def Mult_Two(x):
    
    """If x is a multiple of 2 then return x*2 else return x"""
    
    if x%2 ==0:
        return x*2
    else:
        return x
    
#Call the function
Mult_Two(10)

20

In [8]:
#Equivalent lambda function
f = lambda x: x*2 if x%2==0 else x

#Call the function
f(10)

20

These lambda functions are very useful when used with apply to create custom columns. Let's have a look again at the mtcars:

In [10]:
import pandas as pd
from pandasql import sqldf
sqldf("SELECT AVG(mpg) as avg_mpg, cyl from df_mtcars GROUP BY cyl")


,avg_mpg,cyl
0,26.663636,4
1,19.742857,6
2,15.100000,8


In [1]:
import pandas as pd

df_mtcars = pd.read_csv("Data/mtcars.csv")

df_mtcars.head()

,car_name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


Let's say I want to count how many cars have a number in its name.  First I will create a binary column that is 1 in rows where the car name has a number and zero otherwise. We will use a lambda combined with an apply to create this column. Then we will just take a sum of this new column.

In [14]:
#Create new binary column
#The isalpha() string method return True is the string is all letters and False otherwise.
df_mtcars["num_in_name"] = df_mtcars.car_name.apply(lambda x: 1 if x.replace(" ", "").\
                                                    isalpha() else 0)

df_mtcars.head(15)

,car_name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,num_in_name
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,0
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,0
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,0
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,0
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,1
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,1
6,Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4,0
7,Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2,0
8,Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2,0
9,Merc 280,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4,0


In [13]:
#Take the sum
df_mtcars.num_in_name.sum()

15

The lambda function above will be called once for each row, where the input x will take the value of car_name for the given row.  

Next, let's see how we can use a lambda function with an apply across multiple columns.  Let's say cars are environmentally friendly if they have mpg >=20 if they are 4 or 6 cylinders and mpg>=16 if they are 8 cylinders. We will use apply to create a binary column that takes value 1 in rows that correspond to environmentally friendly cars and 0 otherwise.

In [25]:
#Create new column
df_mtcars["env_friendly"] = df_mtcars.apply(lambda row: 1 if (row["cyl"] in [4,6] and row["mpg"]>=20)\
                                            or (row["cyl"]==8 and row["mpg"]>=16) else 0,\
                                            axis = 1)
df_mtcars.head()

,car_name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,num_in_name,env_friendly
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,0,1
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,0,1
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,0,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,0,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,1,1


The above lambda function is called once per row where the input row take the form of a series where the column names become the index.

One thing to note: unfortunately you cannot have an if/elif/esle statement inside a lambda function.

## Filtering Rows

In this section, we will see how to select only the rows of a dataframe that satisfy certain condition(s).  This is akin to a WHERE statement in SQL.

Let's work with only the first 7 rows of mtcars

In [26]:
#Cut mtcars df
df_mtcars_small = df_mtcars.head(7)

df_mtcars_small

,car_name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,num_in_name,env_friendly
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,0,1
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,0,1
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,0,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,0,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,1,1
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,1,0
6,Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4,0,0


Let's say I only want to look at cars that have >=150 hp.  First I will show the command and then I will break it down.

In [27]:
#Picking out rows that only correspond to cars with >=150 hp + all columns
fast_cars  = df_mtcars_small.loc[df_mtcars_small.hp >= 150, :]

fast_cars

,car_name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,num_in_name,env_friendly
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.44,17.02,0,0,3,2,1,1
6,Duster 360,14.3,8,360.0,245,3.21,3.57,15.84,0,0,3,4,0,0


We can execute this query in two steps to see exactly what is happening under the hood.

In [30]:
#The inner statement returns a boolean seris
boolean_series = df_mtcars_small.hp >= 150

boolean_series

0    False
1    False
2    False
3    False
4     True
5    False
6     True
Name: hp, dtype: bool

In [31]:
#And we could then pass in this series into the first argument of loc.
df_mtcars_small.loc[boolean_series,:]


,car_name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,num_in_name,env_friendly
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.44,17.02,0,0,3,2,1,1
6,Duster 360,14.3,8,360.0,245,3.21,3.57,15.84,0,0,3,4,0,0


Let's say I want to select cars with am = 1, but for these cars I only want to look at the columns mpg, hp, am.  I can do this in the exact same way that we select specific columns with .loc - just specify the list of column names that you want.

In [32]:
#Select our cars with am = 1 and only look at columns mpg, hp, am
df_mtcars_small.loc[df_mtcars_small.am == 1, ["mpg", "hp", "am"]]

,mpg,hp,am
0,21.0,110,1
1,21.0,110,1
2,22.8,93,1


Now let's see how we can filter on multiple condition.  For an "and" clause use & and for an "or" clause use |. Make sure to put each clause in parantheses.  Let's say I want to look at the rows of all cars with 6 cylinders and 4 gears. 

In [33]:
#All cars with 4 gears and 6 cylinders + all columns
df_mtcars_small.loc[(df_mtcars_small.cyl == 6) & (df_mtcars_small.gear == 4), :]

,car_name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,num_in_name,env_friendly
0,Mazda RX4,21.0,6,160.0,110,3.9,2.620,16.46,0,1,4,4,0,1
1,Mazda RX4 Wag,21.0,6,160.0,110,3.9,2.875,17.02,0,1,4,4,0,1


Next let's see how to use the "or".  We will select all rows that either have am=0 or qsec>=17.

In [34]:
#All cars with am=0 ot qsec>=17
df_mtcars_small.loc[(df_mtcars_small.am ==0) | (df_mtcars_small.qsec>=17), :]

,car_name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,num_in_name,env_friendly
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,0,1
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,0,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,0,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,1,1
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,1,0
6,Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4,0,0


Finally, let's say you wanted to look at all cars with either 4 or 6 cylinders.  One way to do this is with an "or" statement as follows.

In [35]:
#All cars with 4 ro 6 cylinders
df_mtcars_small.loc[(df_mtcars_small.cyl == 4) | (df_mtcars_small.cyl == 6), :]

,car_name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,num_in_name,env_friendly
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,0,1
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,0,1
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,0,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,0,1
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,1,0


A faster way to accomplish this same filtering is with the isin() method for series', which takes as input a list of values to search for.

In [36]:
#isin() on column cyl
df_mtcars_small.cyl.isin([4,6])

0     True
1     True
2     True
3     True
4    False
5     True
6    False
Name: cyl, dtype: bool

Since we get back a boolean series with a True for cars that are either 4 or 6 cylinders and a False otherwise we can use isin() inplace of a bunch of ors.

In [38]:
#Using isin() to look at 4 and 6 cylinder cars
df_mtcars_small.loc[df_mtcars_small.cyl.isin([4,6]), :]

,car_name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,num_in_name,env_friendly
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,0,1
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,0,1
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,0,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,0,1
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,1,0


## Miscellaneous 

In this miscellaneous section, you will see additional uses of string methods on series objects as well as how to deal wth duplicate rows.

Recall that to invoke string methods on a series, wen first need a .str.  Let's say I want to get the first word in each car name using string method. Let's first see what happens if we first just split on a blank space.

In [48]:
#Remind what df_mtcars_small looks like
df_mtcars_small

,car_name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,num_in_name,env_friendly
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,0,1
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,0,1
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,0,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,0,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,1,1
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,1,0
6,Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4,0,0


In [42]:
#Split on blank space - We get back a series of lists.
parsed_names = df_mtcars_small.car_name.str.split(" ")

parsed_names

0            [Mazda, RX4]
1       [Mazda, RX4, Wag]
2           [Datsun, 710]
3      [Hornet, 4, Drive]
4    [Hornet, Sportabout]
5               [Valiant]
6           [Duster, 360]
Name: car_name, dtype: object

In [43]:
#Let's look at the type of the first element to see that it is a list
type(parsed_names[0])

list

In [45]:
#Now I can use an apply with a lambda function to get the first name
first_names = parsed_names.apply(lambda x: x[0])

first_names

0      Mazda
1      Mazda
2     Datsun
3     Hornet
4     Hornet
5    Valiant
6     Duster
Name: car_name, dtype: object

This accomplishes the task.  Another way to do this is to set expand = True in the split method. This will create a separate column for each element of the list that results from the split.

In [46]:
#Set expand  = True
parsed_names_expand = df_mtcars_small.car_name.str.split(" ", expand = True)

parsed_names_expand

,0,1,2
0,Mazda,RX4,None
1,Mazda,RX4,Wag
2,Datsun,710,None
3,Hornet,4,Drive
4,Hornet,Sportabout,None
5,Valiant,None,None
6,Duster,360,None


In [47]:
#We can now just pull out the first column
first_names_2 = parsed_names_expand.loc[:,0]

first_names_2

0      Mazda
1      Mazda
2     Datsun
3     Hornet
4     Hornet
5    Valiant
6     Duster
Name: 0, dtype: object

Wrapping up this section, we will look at how to deal with duplicate rows.

In [50]:
#Remind what df_mtcars_small looks like
df_mtcars_small

,car_name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,num_in_name,env_friendly
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,0,1
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,0,1
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,0,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,0,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,1,1
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,1,0
6,Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4,0,0


The two main methods that we will use are duplicated() and drop_duplicates().  The former returns a boolean series and the latter can be used to directly delete duplicate rows from a dataframe. For duplicated() method the inputs are:

- keep
    - "first": Mark duplicates as True except for the first occurrence.
    - "last": Mark duplicates as True except for the last occurrence.
    - False: Mark all duplicates as True
    
For the drop_duplicates() method the keep arguments does the following.

- keep
    - "first": Drop duplicates except for the first occurrence.
    - "last": Drop duplicates except for the last occurrence.
    - False: Drop all duplicates
    
The second arguments for both is:

- subset: Only consider certain columns for identifying duplicates. If subset is not specific,  by default all of the columns will be used.

Let's get a dataframe with just cyl and the gear columns.

In [53]:
#Pick out just cyl and gear columns
df_cyl_gear = df_mtcars_small.loc[:,["cyl", "gear"]]

df_cyl_gear

,cyl,gear
0,6,4
1,6,4
2,4,4
3,6,3
4,8,3
5,6,3
6,8,3


In [56]:
# with keep = "first" - mark duplicates as True except for the first one
duplicates_first = df_cyl_gear.duplicated(keep = "first")

duplicates_first

0    False
1     True
2    False
3    False
4    False
5     True
6     True
dtype: bool

In [67]:
# with keep = "last" - mark duplicates as True except for the last one
duplicates_last = df_cyl_gear.duplicated(keep = "last")

duplicates_last

0     True
1    False
2    False
3     True
4     True
5    False
6    False
dtype: bool

In [60]:
#We can then slice using this series to get all unique rows
df_cyl_gear.loc[duplicates_first == False,:]

,cyl,gear
0,6,4
2,4,4
3,6,3
4,8,3


In [1]:
#We can also use duplicated() to count the number of repeated rows
duplicates_first.sum()

NameError: name 'duplicates_first' is not defined

The drop_duplicates() methods will do the deletion for us in one step.

In [64]:
#Using drop_duplicates() method
df_mtcars_small.drop_duplicates(keep = "first", subset=["cyl", "gear"],\
                                inplace = False)

,car_name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,num_in_name,env_friendly
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,0,1
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,0,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,0,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,1,1


In the above example, we get back all of the columns.  We could further slice the dataframe that is returned to get just the cylinder and gear columns.

In [65]:
#Using drop_duplicates() method
df_mtcars_small.drop_duplicates(keep = "first", subset=["cyl", "gear"],\
                                inplace = False)[["cyl", "gear"]]

,cyl,gear
0,6,4
2,4,4
3,6,3
4,8,3
